In [59]:
# !pip install pandas

In [60]:
import openpyxl
from openpyxl import Workbook
from copy import deepcopy
from typing import Iterable
from collections import defaultdict
import pandas as pd

In [2]:
## 검색할 파일명 기입 ##
파일명 = "Building List Resource_20230110.xlsx"

In [62]:
df = pd.read_excel(파일명,"EGAT")

In [63]:
df

,[AR] Building List (Issued),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,As of 2022-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No.,Unit,Project\nFacility\nCode,Building Tag,Database Code,Building Name,Building/Shelter,NaN,NaN,Green/\nBrown,...,NaN,NaN,NaN,NaN,Blast Design,NaN,Fire\nProofing,OHC/Hoist,Sanitary,Remark
2,NaN,NaN,NaN,NaN,NaN,NaN,Type,Superstructure,Project WBS,NaN,...,Area\nDeduction\n(m2),Including\nTotal Area\n(m2),Total Area\n(m2),Volume\n(m3),Pso\n(psi),Duration\n(ms),NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,998_Other Buildings and Project Overall Item,998,1000-OB-45C,A102,Operator Building (PDH),BD,RC,Building(RC),Green,...,NaN,NaN,20.25,104.29,NaN,NaN,NaN,NaN,O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,NaN,NaN,NaN,NaN,NaN,NaN,BD (Building) :,NaN,NaN,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,NaN,NaN,NaN,NaN,NaN,NaN,OS (Open Shelter) :,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,NaN,NaN,NaN,NaN,NaN,NaN,PS (Partially Open Shelter) :,NaN,NaN,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,NaN,NaN,NaN,NaN,NaN,NaN,FD (Foundation) :,NaN,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def set_search_engine(filename, header_identifier):
    doc = openpyxl.load_workbook(f'{파일명}')
    
    return doc

search_engine = set_search_engine(파일명, "Name")
search_engine.sheetnames
sheet = search_engine["EGAT"] 

In [16]:
db_ = [list(sheet.values)[2], list(sheet.values)[3], list(sheet.values)[4]]
db = list(zip(*db))
db

[('No.', None, None),
 ('Unit', None, None),
 ('Project\nFacility\nCode', None, None),
 ('Building Tag', None, None),
 ('Database Code', None, None),
 ('Building Name', None, None),
 ('Building/Shelter', 'Type', None),
 (None, 'Superstructure', None),
 (None, 'Project WBS', None),
 ('Green/\nBrown', None, None),
 ('NOS', None, None),
 ('Size', 'Story', 'Total'),
 (None, None, '(Count)'),
 (None, 'Floor', 'Floor'),
 (None, None, 'Info.'),
 (None, 'W\n(m)', None),
 (None, 'L\n(m)', None),
 (None, 'H\n(m)', None),
 (None, 'Area\n(m2)', None),
 (None, 'Area\nDeduction\n(m2)', None),
 (None, 'Including\nTotal Area\n(m2)', None),
 (None, 'Total Area\n(m2)', None),
 (None, 'Volume\n(m3)', None),
 ('Blast Design', 'Pso\n(psi)', None),
 (None, 'Duration\n(ms)', None),
 ('Fire\nProofing', None, None),
 ('OHC/Hoist', None, None),
 ('Sanitary', None, None),
 ('Remark', None, None)]

In [17]:
def findNotNone(row):
    """엑셀의 한행 데이터 중, None이 아닌 항목 찾기"""
    return list(filter(lambda x: x != None, row))

def countNotNone(vals):
    """엑셀의 한행 데이터 중, None이 아닌 항목 개수 세기"""
    return list(map(len, (map(findNotNone, vals))))

In [18]:
help(countNotNone)

Help on function countNotNone in module __main__:

countNotNone(vals)
    엑셀의 한행 데이터 중, None이 아닌 항목 개수 세기



In [101]:
def find_notNone_idxs(row):
    return map(lambda x: x[0], filter(lambda x: x[1]!=None, enumerate(row)))

def group_by_idxs(row):
    idxs = find_notNone_idxs(row)
    db = list(enumerate(idxs))
    res = []
    for i,v in db:
        tmp = []
        if i+1 < len(db):
            tmp.append(list(range(v,db[i+1][1])))
        else:
            tmp.append([v])
        res.append(tmp)
    return list(zip(*res))[0]

idxs = list(find_notNone_idxs(db_[0]))

group_by_idxs(db_[0])

([0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6, 7, 8],
 [9],
 [10],
 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
 [23, 24],
 [25],
 [26],
 [27],
 [28])

In [72]:
def the_func(db):
    depth = len(db)
    db_tr = list(zip(*db))
    def find_notNone_idxs(row):
        return filter(lambda x: x[1]!=None, enumerate(row))
        
    
    lvs = range(depth)
    res2 = []
    for n in lvs:
        res = []
        db_idxs = list(enumerate(db[n]))
        copy = deepcopy(db_idxs)
        for i,v in copy:
            if v != None:
                tmp = [(v)]
            else:
                copy[i] = (copy[i-1])
                tmp.append((copy[i]))
            res.append(copy)
        res2.append(res)
        
        
        
    return res

In [53]:
list(the_func(db_))

[[(28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (11, 'Total'),
  (12, '(Count)'),
  (13, 'Floor'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.')],
 [(28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (11, 'Total'),
  (12, '(Count)'),
  (13, 'Floor'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.'),
  (14, 'Info.')],
 [(28, None),
  (28, None),
  (28, None),
  (28, None),
  (28, None),
  (2

In [20]:
def make_sequential_group(nums):
    """리스트 중 연속한 숫자들을 서브리스트로 구성한다"""
    res = []
    tmp = []
    enum_list = list(enumerate(nums))
    start_i,start_e = enum_list[0]
    for i,e in enum_list:
        if (e - start_e) == (i-start_i):
            if i == len(nums)-1:
                tmp.append(e)
                res.append(deepcopy(tmp))
            else:
                tmp.append(e)
                
        elif (e - start_e) != (i-start_i):
            if i == len(nums)-1:
                res.append(deepcopy(tmp))
                tmp.clear()
                tmp.append(e)
                res.append(deepcopy(tmp))
            else:
                res.append(deepcopy(tmp))
                tmp.clear()
                start_i,start_e = i,e
                tmp.append(e)
        else:
            res.append(deepcopy(tmp))

    return res

In [44]:
def find_MergedArea(rowObjs, colObj):
    """엑셀칼럼객체에서 병합된 셀의 영역idx를 찾는다"""
    startRow_idx = find_header_startRow_idx(rowObjs)
    all_idx_merged =  list(filter(lambda x: x > startRow_idx,
                             (map(lambda x: x[0],
                                  (filter(lambda x: "Merged" in str(x),
                                          enumerate(colObj)))))))
    all_idx_merged.insert(0, startRow_idx)
    sequential_group = make_sequential_group(all_idx_merged)

    return sequential_group

In [48]:
def find_header_startRow_idx(rowObjs):
    """None이 아닌 유효 데이터가 갑자기 많아지는 Row idx 찾기(헤더의 시작)"""
    vals = [[x.value for x in y] for y in rowObjs]
    validData_count = countNotNone(vals)
    
    for i,j in enumerate(validData_count):
        if j - (validData_count[i-1]) >= 5:
            return i

In [49]:
def find_header_endRow_idx(rowObjs, colObjs, startRow_idx):
    """헤더의 끝 row idx 찾기"""
    MergedAreas = list(map(lambda x: find_MergedArea(rowObjs,x), colObjs))
    firstElems_MaybeHeader = map(lambda x: x[0], MergedAreas)
    last_idxs = map(lambda x: x[-1], firstElems_MaybeHeader)
    
    return max(last_idxs)

In [174]:
def fill_missing_h1(headers):
    """find_header_objs 함수 호출 시, 헤더 최상위 레벨의 병합부분 누락 데이터 채움용"""
    new_h1 = deepcopy(headers[0])
    for i,v in enumerate(headers[0]):
        if v == None:
            new_h1[i] = new_h1[i-1]
    headers[0] = new_h1
    return headers

def find_header_objs(rowObjs, colObjs):
    """헤더 startRow_idx와 endRow_idx를 기반으로 헤더객체들 추출 """
    startRow_idx = find_header_startRow_idx(rowObjs)
    endRow_idx = find_header_endRow_idx(rowObjs, colObjs, startRow_idx)
    range_ = range(startRow_idx, endRow_idx+1)
    find_header = lambda rowObj: map(lambda x: x.value, rowObj)
    all_headers = list(map(lambda x: list(find_header(rowObjs[x])), range_))
    all_headers_h1_filled = fill_missing_h1(all_headers)
    zip_all_headers = zip(*all_headers_h1_filled)    
    
    return zip_all_headers

In [175]:
def find_targetColumn_idx(rowObjs, colObjs, _target):
    """타겟 문자열이 포함된 헤더가 위치한 칼럼 인덱스 추출"""
    headers = list(find_header_objs(rowObjs, colObjs))
    headers_with_idx = list(enumerate(headers))
    idx = list(map(lambda x: x[0], headers_with_idx))
    str_headers = list(map(lambda x: str(x[1]), headers_with_idx))
    str_headers_with_idx = list(zip(idx, str_headers))
    
    targets = _target.split()
    
    def is_all_include(targets, strs):
        res = []
        for i in targets:
            if i in strs:  res.append(True)
            else        :  res.append(False)
        return all(res)
    
    find_f = lambda y: list(filter(lambda x: is_all_include(y ,x[1]), str_headers_with_idx))
    
    out = find_f(targets)
    
    return out

---
<br>

---

In [54]:
## 검색할 파일명 기입 ##
파일명 = "Building List Resource_20230110.xlsx"

In [31]:
def set_search_engine(filename, header_identifier):
    doc = openpyxl.load_workbook(f'{파일명}')
    
    return doc

In [265]:
search_engine = set_search_engine(파일명, "Name")
search_engine.sheetnames

['EGAT',
 'HDO RVMP 2020TA',
 'HDO RVMP',
 'HDO SDA',
 'Vistok',
 'IKAN',
 'UTCC',
 'KLNG',
 'SGPC',
 'TACE',
 'HPP4',
 'PTT LNG',
 'LINE',
 'HNCC PKG-1',
 'HNCC PKG-2',
 'HNCC PKG-3',
 'P-GMTP3',
 'MMCC',
 'IKCP',
 'TVEP']

In [266]:
sheet = search_engine["TACE"]  # 임시로  search_engine이 doc 역할( 테스트 후 리팩토링 시 주석 삭제 예정)
sheet

<Worksheet "TACE">

In [267]:
def seach_in_targetSheet(sheet, search_target, target_prop):
    sheet_name = sheet.title
    lookup_target = "Building Name"  ##테스트 후 엔진객체에서 추출하는 걸로 수정
    rowObjs = list(sheet.rows)
    colObjs = list(sheet.columns)
    head_startRow_idx = find_header_startRow_idx(rowObjs)
    head_endRow_idx = find_header_endRow_idx(rowObjs, colObjs, head_startRow_idx)
    lookup_target_col_idx = find_targetColumn_idx(rowObjs, colObjs, lookup_target)[0][0]
    
    f = lambda y: list(map(lambda x: x.value, colObjs[lookup_target_col_idx]))[y+1:]
    data_NoneStr = list(map(str, f(head_endRow_idx)))
    all_target_data = list(filter(lambda x: x[1] != 'None', enumerate(data_NoneStr)))
    
    search_target_data = list(filter(lambda x: search_target in x[1], all_target_data))
    search_target_row_idxs = list(map(lambda x: x[0]+head_endRow_idx+1,search_target_data))
    search_target_col_idxs = find_targetColumn_idx(rowObjs, colObjs, target_prop)
    search_target_col_idx = search_target_col_idxs[0][0]  ## 임시로 첫째항목 기준 데이터 추출로 구현함(추후 수정예정)
    
    def find_target_prop_data(rowObjs, row_idx, col_idx):
        return list(map(lambda x: x.value, rowObjs[row_idx]))[col_idx]
    
    goal_data = list(map(lambda x: find_target_prop_data(rowObjs, x, search_target_col_idx), search_target_row_idxs))
    goal_data_identi = list(map(lambda x: x[1] ,search_target_data))
    goal_data_pair = list(zip(goal_data_identi, goal_data))
    
    result_dict = {sheet_name: goal_data_pair}
    
    return result_dict

In [271]:
seach_in_targetSheet(sheet, "Shelter", "Area")

{'TACE': [(' Co-Catalyst Shelter', '=22.8*6.5'),
  (' Communication center &\n Air Raid Shelter & Training Building',
   '=55.55*14'),
  (' Sea water Chlorination Shelter', '=7.1*8'),
  (' Sea water Filtration Shelter', '=8.6*5.4')]}